#### BaseModel — the DTO base class

In [2]:
from pydantic import BaseModel, EmailStr

class User(BaseModel):
    name: str
    age: int
    email: EmailStr | None = None

u = User(name="Navneet", age="42")
print(u.age)          # 42  (string → int auto-converted)
print(type(u.age))  
print(type(u.email))  

u = User(name="Navneet", age=42, email="nav@xyz.com")
print(u.age)          # 42  (string → int auto-converted)
print(type(u.age))  
print(type(u.email))  

# u = User("Navneet", 42, "nav@xyz.com") => have to use named
# print(u.age)          # 42  (string → int auto-converted)
# print(type(u.age))  
# print(type(u.email))  

42
<class 'int'>
<class 'NoneType'>
42
<class 'int'>
<class 'str'>


#### Field — constraints + metadata

Enforced rules
- min_length, max_length| gt, ge, lt, le | regex |default, default_factory| description (Swagger docs)

In [3]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(min_length=2, max_length=50)
    age: int = Field(gt=0, lt=120)

class Order(BaseModel):
    qty: int = Field(gt=0)
    price: float

o = Order(qty="1", price="112.8")
print(o)

qty=1 price=112.8


#### Inheritance

In [4]:
class UserBase(BaseModel):
    name: str
    email: EmailStr
    nickname: str | None = None

class UserCreate(UserBase):
    pass

class UserOut(UserBase):
    id: int



createuser = UserCreate(name="navneet", email="xyz@gmail.com")
print(createuser)

userout = UserOut(name="navneet", email="xyz@gmail.com", id=100)
print(userout)
userout.model_dump_json()
userout.model_dump(exclude_unset=True)

name='navneet' email='xyz@gmail.com' nickname=None
name='navneet' email='xyz@gmail.com' nickname=None id=100


{'name': 'navneet', 'email': 'xyz@gmail.com', 'id': 100}

#### PATCH BEHAVIOUR

In [5]:
class UserUpdate(BaseModel):
    name: str | None = None
    email: str | None = None
    age: int | None = None

user = UserUpdate(email="old@xyz.com")
all = user.model_dump(exclude_unset=False)
patch = user.model_dump(exclude_unset=False)
print(all)
print(patch)

{'name': None, 'email': 'old@xyz.com', 'age': None}
{'name': None, 'email': 'old@xyz.com', 'age': None}


In [6]:
from typing import Optional

class CustomerBase(BaseModel):
    name: str = Field(min_length=1, max_length=80)
    email: EmailStr

class CustomerCreate(CustomerBase):
    pass  # same fields as base

class CustomerUpdate(BaseModel):
    # partial update (PATCH-like); all optional
    name: str | None = Field(default=None, min_length=1, max_length=80)
    email: EmailStr | None = None

class CustomerOut(CustomerBase):
    id: int

data = {"name": "Navneet", "email": "nav@xyz.com"}
cust = CustomerOut(id = 1, **data)
print(cust)

def greet(name, age):
    print(name, age)


name='Navneet' email='nav@xyz.com' id=1


In [ ]:
from fastapi import FastAPI, HTTPException, status, APIRouter, Depends
class Customer(BaseModel):
    id: int
    name: str
    email: EmailStr
existing = Customer(id=1, name="Navneet", email="old@xyz.com")
existing

def require_api_key():pass
def check_tenant():pass
def rate_limit():pass

custRouter = APIRouter(prefix="", tags=["test"], dependencies=[
    Depends(require_api_key),
    Depends(check_tenant),
    Depends(rate_limit),
])

Customer(id=1, name='Navneet', email='old@xyz.com')

In [8]:
patch = {"email": "new@xyz.com"}

updated = existing.model_copy(update=patch)
updated

Customer(id=1, name='Navneet', email='new@xyz.com')